# Домашнее задание по NLP # 2 [100 баллов]
Составление словарей для классификации по тональности

При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: 
- во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), 
- во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: “теплое пиво” – это плохо, а “теплый свитер” – это хорошо.

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).

Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).



Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
Обучите по коллекции предложений word2vec

Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
 - тест на определение ближайших слов
 - тест на аналогии (мужчина – король : женщина – королева)
 - тест на определение лишнего слова.

Постройте несколько визуализаций:
 - TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
 - задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [5]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [6]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:48, 1857.37it/s]


#### Лемматиризируем

In [9]:
df = pd.DataFrame(responses)

In [10]:
sample = df[:100]

In [11]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

In [12]:
from nltk.corpus import stopwords

mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 'г', 'года', 'руб', 'рублей', 'сумму']

In [13]:
def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

In [14]:
from pymystem3 import Mystem

In [15]:
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [16]:
mystoplemmas = ['день', 'данный', 'который', 'ваш', 'свой', 'месяц', 'какой']
def remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

In [17]:
sample.text = sample.text.str.lower()
sample.text = sample.text.apply(words_only)
sample.text = sample.text.apply(remove_stopwords) 
sample.text = sample.text.apply(lemmatize)
sample.text = sample.text.apply(remove_stoplemmas) 

C:\Users\Nikita\.conda\envs\tensorflow\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


#### Загрузим модель

In [18]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x2b5a1098b48>)

In [19]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

#### Токенизируем и обучаем модель word2Vec

In [20]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from string import punctuation
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

In [22]:
from tqdm import tqdm

tokens = []

for index, row in tqdm(sample.iterrows(), total = len(sample)):
    tokens += tokenizer.tokenize(row['text'].strip())
    #tokens += tokens[0].split()

100%|██████████| 100/100 [00:00<00:00, 4070.64it/s]


In [23]:
for i in range(0,len(tokens)):
    tokens[i] = tokens[i].split()

In [24]:
print("Training model...")

%time model_rus = word2vec.Word2Vec(tokens, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training model...
Wall time: 90.7 ms


#### Проводим несколько тестов:

In [25]:
print(model_rus.wv.most_similar(positive=["карта", "кредит"], negative=["сказать"], topn=1))
print(model_rus.wv.most_similar("отделение", topn=3))
print(model_rus.wv.most_similar("банк", topn=3))
print(model_rus.wv.doesnt_match("открыть счет банк кредит".split()))

[('банк', 0.9999327659606934)]
[('банк', 0.9999404549598694), ('сказать', 0.9999336004257202), ('счет', 0.9999308586120605)]
[('счет', 0.9999755620956421), ('кредит', 0.9999732375144958), ('сказать', 0.9999697208404541)]
кредит


C:\Users\Nikita\.conda\envs\tensorflow\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


#### Сохраняем модель:

In [26]:
model_path = "movie_reviews.model"

print("Saving model...")
model_rus.save(model_path)

Saving model...


In [27]:
model = word2vec.Word2Vec.load(model_path)

#### Визуализируем:

In [90]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []

fd = FreqDist()
for s in tqdm(tokens):
    fd.update(s)

for w in fd.most_common(100):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = model[top_words]

C:\Users\Nikita\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



['банк', 'карта', 'кредит', 'мой', 'клиент', 'счет', 'деньги', 'договор', 'сотрудник', 'сказать', 'платеж', 'вопрос', 'кредитный', 'приходить', 'получать', 'обращаться', 'заявление', 'такой', 'офис', 'время', 'средство', 'просить', 'ситуация', 'сумма', 'отделение', 'мочь', 'являться', 'страховка', 'проблема', 'работа', 'почему', 'сделать', 'понимать', 'отвечать', 'вносить', 'услуга', 'написать', 'звонок', 'комиссия', 'оплата', 'вклад', 'хотеть', 'очень', 'банка', 'заявка', 'звонить', 'срок', 'номер', 'условие', 'говорить']


C:\Users\Nikita\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [91]:
top_words_vec = model[top_words]

C:\Users\Nikita\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [92]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

Wall time: 366 ms


In [93]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (rus model, top100 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

#### Попробуем снизить размерность при помощи SVD:

In [32]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

In [33]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (rus model, top100 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

#### Задаем новое пространство:

- задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

https://stackabuse.com/python-for-nlp-working-with-facebook-fasttext-library/ 

In [94]:
from gensim.models import FastText

model_ru = FastText(tokens, size=100, window=5, min_count=5, workers=4,sg=1)

In [100]:
good = model_ru.wv.get_vector("хорошо")
bad = model_ru.wv.get_vector("плохо")

x_graph = good - bad

In [101]:
fast = model_ru.wv.get_vector("быстро")
slow = model_ru.wv.get_vector("медленно")

y_graph = fast - slow

In [102]:
sberbank_x = model_ru.wv.get_vector('сбербанк') * x_graph
sberbank_y = model_ru.wv.get_vector('сбербанк') * y_graph

In [103]:
tinkoff_x = model_ru.wv.get_vector('тинькофф') * x_graph
tinkoff_y = model_ru.wv.get_vector('тинькофф') * y_graph 

In [106]:
### не пойму как использовать вектора банков в визуализации

In [104]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (rus model, top100 words, +SVD)")

source = ColumnDataSource(data=dict(x1=x_graph,
                                    x2=y_graph,
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...